## 21. Dependency direction & inversion (passing in a collaborator)

High‑level components shouldn’t depend on low‑level concrete details. Instead, both depend on **abstractions** (a principle known as *Dependency Inversion*).  

Practical recipe in Python:
1. Define a small interface (protocol / ABC) describing what you need.  
2. Pass an implementation to your class/function via constructor or argument.  

Benefits:
• Swap implementations (e.g., real DB ↔ in‑memory fake) without code changes.  
• Breaks import cycles, lowering coupling.  
• Tests inject stubs/mocks easily.

Common gotcha → service classes that *new‑up* their own dependencies inside `__init__`, making them hard to fake during tests.

```python
class INotifier:
    def send(self, msg: str) -> None: ...

class EmailNotifier(INotifier):
    def send(self, msg):
        print(f'Email: {msg}')

class SignupService:
    def __init__(self, notifier: INotifier):
        self.notifier = notifier  # injected dependency
    def signup(self, user):
        # save user ...
        self.notifier.send('welcome')

SignupService(EmailNotifier()).signup('alice')
```

### Quick check

1. True / False Creating `EmailNotifier()` *inside* `SignupService` aids testability.

2. A key benefit of dependency inversion is:
  a. Faster execution   b. Swappable implementations

<details><summary>Answer key</summary>

1. **False** — hard‑codes concrete class, blocking injection.
2. **b**.

</details>

## 22. Interfaces / protocols for swappable components

An **interface** captures a *behavior contract* without tying you to a concrete type.  
In Python ≥3.8, use `typing.Protocol` for lightweight structural interfaces (duck typing).  

Why:
• Enables static checking (`mypy`) that replacements actually fit.  
• Documents expectations for future maintainers.  
• Power pattern for plugin systems.

Gotcha: wide ‘God’ interfaces—keep only the methods truly required by callers.

```python
from typing import Protocol

class Storage(Protocol):
    def put(self, key: str, data: bytes) -> None: ...
    def get(self, key: str) -> bytes | None: ...

class MemoryStorage(dict):
    def put(self, k, d): self[k] = d
    def get(self, k):    return self.get(k)

def cache_avatar(storage: Storage, user_id: str, img: bytes):
    storage.put(user_id, img)

cache_avatar(MemoryStorage(), '42', b'raw')
```

### Quick check

1. `Protocol` checks happen:
  a. at runtime   b. during static analysis

2. True / False A concrete class must explicitly subclass `Protocol` to conform.

<details><summary>Answer key</summary>

1. **b** — via tools like mypy/pyright.
2. **False** — structural conformity suffices.

</details>

## 23. Basic refactor moves: extract function, inline variable, rename symbol

Refactoring = **changing code structure without changing behavior**.  Three low‑risk moves:
• **Extract Function** – break a long block into a named function; improves reuse & readability.  
• **Inline Variable** – remove needless temp vars that add no meaning.  
• **Rename Symbol** – choose clearer names, guided by tests & IDE rename‑all tools.

Always run tests before & after to ensure behaviour unchanged.

```python
# before
tax = price * 0.07
total = price + tax

# inline variable
total = price * 1.07

# extract function
def price_with_tax(p):
    return p * 1.07
total = price_with_tax(price)
```

### Quick check

1. True / False Refactoring should be done only when adding new features.

2. Which refactor risks changing external API?
  a. rename internal var   b. extract private helper

<details><summary>Answer key</summary>

1. **False** — can be purely cleanup.
2. **a** if the variable is part of public interface.

</details>

## 24. Writing *pure functions* and isolating side‑effects

A **pure function**: output depends *only* on inputs; no external state mutated, no I/O.  
Side‑effects (logs, DB writes) should live at the edges of the system.  Benefits:
• Easy unit tests — pass inputs, assert output.  
• Naturally thread‑safe and cache‑friendly.

Strategy: keep core logic pure, wrap with thin impure adapter layer.

```python
def pure_discount(price: float, pct: float) -> float:
    return price * (1 - pct)

def apply_discount_and_log(price, pct, logger):
    new_price = pure_discount(price, pct)   # pure core
    logger.info('discount applied')         # side‑effect at boundary
    return new_price
```

### Quick check

1. Pure functions help with:
  a. deterministic tests  b. file DB writes

2. True / False Random number generation keeps a function pure.

<details><summary>Answer key</summary>

1. **a**.
2. **False** — randomness makes output non‑deterministic.

</details>

## 25. Memoisation & caching as performance straps

If a pure (or effectively pure) function is **expensive** and called repeatedly with the same inputs, store previously computed results → instant lookup.  Python ships `functools.lru_cache`.

Be mindful of:
• Cache key = function args; mutable/unhashable args need transformation.  
• Eviction policy and memory footprint.
• Stale data for non‑pure functions.

```python
from functools import lru_cache
import math, time

@lru_cache(maxsize=128)
def slow_sin(x):
    time.sleep(0.1)
    return math.sin(x)

for _ in range(2):
    start=time.perf_counter(); slow_sin(1.23); print(time.perf_counter()-start)
```

### Quick check

1. `lru_cache` identifies results using:
  a. object identity  b. function arguments value

2. True / False Caching impure functions can yield inconsistent behaviour.

<details><summary>Answer key</summary>

1. **b** — args must be hashable.
2. **True**.

</details>